In [1]:
import torch
from torchvision.models import resnet18, resnet50
import model_utils
import plotter
import data_utils
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader
import importlib
import numpy as np
import pandas as pd
import torchvision.transforms as transforms


model_utils.fix_random(42)

In [2]:
dataset = data_utils.RVAADataset(root_dir="data/RealVsAiArt")
df_stats = data_utils.get_dataset_stats(dataset)
pd.DataFrame.from_dict(df_stats, orient='index')

,0
min_width,183.000000
max_width,8495.000000
mean_width,1240.438144
std_width,865.566509
min_height,135.000000
max_height,7091.000000
mean_height,941.283505
std_height,718.557426


In [3]:
transform = transforms.Compose([
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.Resize(128),            # Resize the shorter side to 128 while keeping aspect ratio
    transforms.Lambda(data_utils.pad_to_square),  # Apply the dynamic padding to make the image square
    transforms.Resize((128, 128)),     # Ensure the final image is exactly 128x128
    transforms.ToTensor()              # Convert the image to a tensor
])

dataset = data_utils.RVAADataset(root_dir="data/RealVsAiArt", transform=transform)

LEARNING_RATE = 3e-4
EPOCHS = 10
BATCH_SIZE = 32

train_dataset, val_dataset, test_dataset = data_utils.train_test_split((0.7, 0.10), dataset=dataset)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

loss_fn = nn.BCEWithLogitsLoss()
device = model_utils.get_device()
print(f"using {device} device")

dataloaders = (train_dataloader, val_dataloader)

using cuda device


In [4]:
ft_resnet50 = resnet50(weights='IMAGENET1K_V1')
model_utils.prepare_for_ft(ft_resnet50, num_classes=1)
ft_resnet50.to(device)

optimizer = torch.optim.AdamW(ft_resnet50.fc.parameters(), lr=LEARNING_RATE)

In [5]:
importlib.reload(model_utils)
artifacts = model_utils.train(dataloaders, loss_fn, optimizer, ft_resnet50, "FTRes50_newdata", BATCH_SIZE, EPOCHS, loss_thresh=2.5, force_train=True)

Epoch 1/10
-------------------------------


c:\Users\ASUS\Desktop\ethics_2\ethics_project2\.venv\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Validation Error:
Accuracy: 51.5%, Avg loss: 0.740563 

new best model found
best model saved
Epoch 2/10
-------------------------------


KeyboardInterrupt: 

In [ ]:
model_utils.test(test_dataloader, ft_resnet50, loss_fn, device)

In [ ]:
if artifacts != None:
    losses, train_accs, test_accs = artifacts
    plotter.plot_metrics((train_accs, test_accs), "accuracy")

GradCAM Test

In [ ]:
import xai

xai.explain_gradcam_single_image(train_dataloader, ft_resnet50, [ft_resnet50.layer1, ft_resnet50.layer2, ft_resnet50.layer3, ft_resnet50.layer4])